In [20]:
from os import listdir
from os.path import isfile, join
from PIL import Image

import random
import math
import time
import numpy as np
import cv2
import os
import shutil

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import torchvision.models.segmentation
import torch
import torchvision.transforms as tf
import torch.nn as nn
import torch.nn.functional as F

In [21]:
widthSteering = heightSteering = 140

widthRoad = 600
heightRoad = 200

modelsPath = 'Models'
modelName = 'bestModel.torch'
pathToModel = f'{modelsPath}/{modelName}'

framesPath = '/media/oliver/ML_Data/FYP/Frames/DrivingFramesStarting'
labeledFrames = '/media/oliver/ML_Data/FYP/Frames/DrivingTrainingFramesPreAugmentation'
augmentedFrames = 'AugmentedFrames'
wheelFrames = 'WheelFrames'

#road crop area
x1R = 400
x2R = 600
y1R = 630
y2R = 1280

#road image size
widthRoad  = y2R - y1R
heightRoad = x2R - x1R

#wheel crop area
x1W = 812
x2W = 932
y1W = 913
y2W = 1033

#wheel image size
widthWheel  = y2W - y1W
heightWheel = x2W-x1W


In [22]:
exist = os.path.exists(modelsPath)
if exist:
    print('Models Path Directoty Exists')
else:
    print('Directory Does not Exists')

exist = os.path.exists(framesPath)
if exist:
    print("Frames Directoty Exists")
else:
    print("Frames Directory Does Not Exists")


exist = os.path.exists(labeledFrames)
if not exist:
    os.makedirs(labeledFrames)
    print("Labled Frames Directoty Created")
else:
    print('Labled Frames Directory Exists')
    #Uncomment these two lines if you want to delete the frames folder before adding more frames
    shutil.rmtree(labeledFrames)
    os.makedirs(labeledFrames)

exist = os.path.exists(augmentedFrames)
if not exist:
    os.makedirs(labeledFrames)
    print("Labled Frames Directoty Created")
else:
    print('Labled Frames Directory Exists')
    #Uncomment these two lines if you want to delete the frames folder before adding more frames
    shutil.rmtree(labeledFrames)
    os.makedirs(labeledFrames)

exist = os.path.exists(wheelFrames)
if not exist:
    os.makedirs(wheelFrames)
    print("Wheel Frames Directoty Created")
else:
    print('Wheel Frames Directory Exists')
    #Uncomment these two lines if you want to delete the frames folder before adding more frames
    shutil.rmtree(labeledFrames)
    os.makedirs(labeledFrames)

Models Path Directoty Exists
Frames Directoty Exists
Labled Frames Directoty Created
Labled Frames Directory Exists
Wheel Frames Directory Exists


In [23]:
startingFrames = [f for f in listdir(framesPath) if isfile(join(framesPath, f))]

In [24]:
len(startingFrames)

56044

In [25]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.fc = nn.Linear(512, 128)
        
        self.branch_a1 = nn.Linear(128, 32)
        self.branch_a2 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = F.leaky_relu(self.fc(x))

        a = F.leaky_relu(self.branch_a1(x))
        
        out1 = self.branch_a2(a)
        
        return out1

In [26]:
# inference frames

#load net
#To Load Pretrained Weights:   weights='ResNet18_Weights.DEFAULT'
resnet18 = torchvision.models.resnet18()
resnet18.fc = nn.Identity()
net_add=net()
model = nn.Sequential(resnet18, net_add)

# Set device GPU or CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 

# load model to GPU
model = model.to(device)

model.load_state_dict(torch.load(pathToModel))

model.eval()
for file in startingFrames:

    noExtension = file[:-4]

    filePath = f'{framesPath}/{file}'

    #load image
    wholeFrame = cv2.imread(filePath)

    #crop to steering wheel logo and transform image to black and white but in RGB color space
    crop = wholeFrame[x1W:x2W, y1W:y2W]

    grayImage = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    (thresh, contrastImg) = cv2.threshold(grayImage, 100, 255, cv2.THRESH_BINARY)
    backtorgb = cv2.cvtColor(contrastImg,cv2.COLOR_GRAY2RGB)


    transformImg=tf.Compose([tf.ToPILImage(),tf.Resize((heightWheel,widthWheel)),tf.ToTensor(),tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) 
    wheel=transformImg(np.array(backtorgb))

    wheel = wheel.to(device).unsqueeze(0)

    with torch.no_grad():
        prediction = model(wheel)  # Run net

    detectedAngle = prediction.data.cpu().numpy()[0][0]

    realAngle = (180*detectedAngle)-90

    cv2.imwrite(f'{wheelFrames}/wheelCrop_angleNorm_{detectedAngle}_angleReal_{realAngle}.jpg',backtorgb)
    
    crop = wholeFrame[x1R:x2R, y1R:y2R]
    resized = cv2.resize(crop, [180,56], interpolation = cv2.INTER_AREA)

    cv2.imwrite(f'{labeledFrames}/{noExtension}_angleNorm_{detectedAngle}.jpg', resized)

In [72]:
from multiprocessing import Process
import multiprocessing
import numpy as np

startingFrames = [f for f in listdir(labeledFrames) if isfile(join(labeledFrames, f))]

frameBlocks = np.array_split(startingFrames, 24)

In [73]:
len(frameBlocks)

24

In [75]:
def run(frames):
    for file in frames:
        noExtension = file[:-4]

        filePath = f'DrivingTrainingFramesPreAugmentation/{file}'
        
        try:
            resized = cv2.imread(filePath)
        except Exception as ex:
            print(ex)
        finally:
            print("Starting Augmentatiion on img")
            cv2.imwrite(f'{labeledFrames}/{detectedAngle}_degrees_roadcrop_alpha_no_beta_no_{noExtension}.jpg',resized)


            #array of all angle values to rotate by
            alphas = np.linspace(0.2,1.2,4)
            betas = np.linspace(0.2,2,4)

            for alpha in alphas:
                for beta in betas:

                    brighness = cv2.convertScaleAbs(resized, alpha, beta)
                    cv2.imwrite(f'{labeledFrames}/{detectedAngle}_degrees_roadcrop_alpha_{alpha}_beta_{beta}_{noExtension}.jpg',brighness)

                    for f in range(0,1):
                        blur = cv2.blur(brighness,(f,f))
                        cv2.imwrite(f'{augmentedFrames}/{detectedAngle}_degrees_roadcrop_alpha_{alpha}_beta_{beta}_filter_{f}_{noExtension}.jpg',blur)

In [76]:
procs = []
for frames in frameBlocks:
    proc = Process(target=run, args=(frames,))
    procs.append(proc)
    proc.start()

for proc in procs:
    proc.join()

Starting Augmentatiion on img


Process Process-423:


Starting Augmentatiion on img


Traceback (most recent call last):


Starting Augmentatiion on img

...

  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
cv2.error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/filter.dispatch.cpp:145: error: (-215:Assertion failed) 0 <= anchor.x && anchor.x < ksize.width && 0 <= anchor.y && anchor.y < ksize.height in function 'init'

cv2.error: OpenCV(4.7.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:783: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

  File "/tmp/ipykernel_89254/3755200731.py", line 27, in run
    blur = cv2.blur(brighness,(f,f))


Process Process-442:
cv2.error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/filter.dispatch.cpp:145: error: (-215:Assertion failed) 0 <= anchor.x && anchor.x < ksize.width && 0 <= anchor.y && anchor.y < ksize.height in function 'init'

Process Process-445:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)


Starting Augmentatiion on img

  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/tmp/ipykernel_89254/3755200731.py", line 27, in run
    blur = cv2.blur(brighness,(f,f))


  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
cv2.error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/filter.dispatch.cpp:145: error: (-215:Assertion failed) 0 <= anchor.x && anchor.x < ksize.width && 0 <= anchor.y && anchor.y < ksize.height in function 'init'

  File "/tmp/ipykernel_89254/3755200731.py", line 13, in run
    cv2.imwrite(f'{labeledFrames}/{detectedAngle}_degrees_roadcrop_alpha_no_beta_no_{noExtension}.jpg',resized)
Process Process-446:
cv2.error: OpenCV(4.7.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:783: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_89254/3755200731.py", line 13, in run
    cv2.imwrit